In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import sys
from loguru import logger
import arrow

sys.path.append("..")
from data import get_config

data_path = Path("../data")

logger.remove()
logger.add(sys.stderr, level="INFO")

2024-01-23 14:00:28.199 | INFO     | data:<module>:15 - data_path: g:\Arvin-Edison WSD-1215\121523003-Frick Unit Pipeline\400 GIS\Scripts\aewsd_frick_unit\util\..\data


2

In [2]:
full_config = get_config("Full")
simple_config = get_config("Simple")
full_config.head(2)

,id,clip_to_district,clip_to_frick,Name,shape_type,color,size,alpha,label,file_path,file_type,shp,gdb,parquet,layer,clip_to_frick_x,full,simple
16,5.0,NaN,NaN,FFPPP Discharge Pipeline,line,#A2678A,4.0,1.0,CANAL,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN,NaN,NaN,1.0,1.0
15,4.0,NaN,NaN,AEWSD North Canal,line,blue,4.0,1.0,CANAL,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN,NaN,NaN,1.0,1.0


In [3]:
district_boundary = gpd.read_file(
    full_config.loc[full_config["Name"] == "District Boundary"]["file_path"].iloc[0]
)
service_boundary = gpd.read_file(
    full_config.loc[full_config["Name"] == "Frick Unit Service Area"]["file_path"].iloc[
        0
    ]
)

intersect = lambda gdf: gdf[gdf.intersects(service_boundary.unary_union)]
clip = lambda gdf: gdf.clip(service_boundary)


def get_layer(row):
    try:
        logger.info(f"Loading layer {row['Name']}")
        file_path = Path(row["file_path"])
        read_options = {
            ".shp": lambda file: gpd.read_file(file),
            ".gdb": lambda file: gpd.read_file(file, layer=row["layer"]),
            ".parquet": lambda file: gpd.read_parquet(file),
        }
        # logger.info(file_path)
        gdf = read_options[file_path.suffix](file_path).to_crs(epsg=2229)

        if row["Name"] == "APNs":
            gdf["label"] = (
                gdf["label"] + "\nService Area Type: " + gdf["Service Area Type"]
            )
            gdf["label"] = gdf["label"].str.replace(r"\n", "<br>", regex=True)

            # gdf = full_gdf[full_gdf['layer']==layer]
        if row["Name"] == "Panama Unit Service Area":
            print(gdf.head(2))
            gdf = gdf.loc[gdf["Name"] != "Frick Unit North Service Area"]

        if row["Name"] == "Panama Unit Pipeline":
            gdf = gdf.loc[gdf["Name"] != "Frick Unit"]

        gdf["color"] = row["color"]
        # gdf["label"] = gdf[row["label"]].astype(str)
        # print(gdf.columns)
        if type(row["label"]) == str:
            gdf["label"] = gdf[row["label"]].astype(str).replace("nan", "")
        else:
            gdf["label"] = ""

        gdf["layer"] = row["Name"]
        gdf["size"] = row["size"]
        if row["clip_to_frick"] == True:
            logger.info(f"Clipping layer {row['Name']}")
            gdf = clip(gdf)

        return gdf[
            ["color", "label", "layer", "size", "geometry"]
        ]  # .to_crs(epsg=4326)
    except Exception as e:
        logger.error(f"Layer {row['Name']} failed to load due to {e}")
        return None

In [4]:
def get_sheet(sheet_name):
    config = full_config if sheet_name == "Full" else simple_config
    gdfs = {y["Name"]: get_layer(y) for i, y in config.iterrows()}
    epsg = 4326
    gdf = pd.concat([gdf.to_crs(epsg=epsg) for gdf in gdfs.values() if gdf is not None])
    gdf.to_parquet(
        data_path / f"{sheet_name}-{arrow.now().format('YYYY-MM-DD')}.parquet"
    )


get_sheet("Full")
get_sheet("Simple")

2024-01-23 14:00:30.817 | INFO     | __main__:get_layer:11 - Loading layer FFPPP Discharge Pipeline
2024-01-23 14:00:30.855 | INFO     | __main__:get_layer:11 - Loading layer AEWSD North Canal
2024-01-23 14:00:30.898 | INFO     | __main__:get_layer:11 - Loading layer District Boundary
2024-01-23 14:00:30.922 | INFO     | __main__:get_layer:11 - Loading layer Sandrini Unit Service Area
2024-01-23 14:00:30.973 | INFO     | __main__:get_layer:11 - Loading layer Sandrini Unit proposed pipeline
2024-01-23 14:00:31.022 | INFO     | __main__:get_layer:11 - Loading layer Groundwater Service Area
2024-01-23 14:00:32.183 | INFO     | __main__:get_layer:11 - Loading layer Panama Unit Service Area
2024-01-23 14:00:32.228 | INFO     | __main__:get_layer:11 - Loading layer Panama Unit Pipeline
2024-01-23 14:00:32.266 | INFO     | __main__:get_layer:11 - Loading layer Existing Laterals
2024-01-23 14:00:32.306 | INFO     | __main__:get_layer:11 - Loading layer DiGiorgio Service Area
2024-01-23 14:00:3

                               Name Descr  Extrude        AltMode  Tessellate  \
0     Frick Unit North Service Area  None        0  ClampToGround           1   
1  Frick Unit South L2 Service Area  None        0  ClampToGround           1   

  Symbology  Transparen  PPGIS_ac  \
0         0          33   2843.79   
1         1          33   1898.78   

                                            geometry  
0  POLYGON Z ((6292789.326 2292680.742 0.000, 629...  
1  POLYGON Z ((6299433.691 2292600.894 0.000, 629...  


2024-01-23 14:00:32.625 | INFO     | __main__:get_layer:11 - Loading layer Landowner Wells
2024-01-23 14:00:33.041 | INFO     | __main__:get_layer:11 - Loading layer Pipeline ESMT
2024-01-23 14:00:33.078 | INFO     | __main__:get_layer:11 - Loading layer Proposed Pipeline
2024-01-23 14:00:33.118 | INFO     | __main__:get_layer:11 - Loading layer Proposed Turnout
2024-01-23 14:00:33.635 | INFO     | __main__:get_layer:11 - Loading layer FFPPP Discharge Pipeline
2024-01-23 14:00:33.655 | INFO     | __main__:get_layer:11 - Loading layer AEWSD North Canal
2024-01-23 14:00:33.677 | INFO     | __main__:get_layer:11 - Loading layer District Boundary
2024-01-23 14:00:33.693 | INFO     | __main__:get_layer:11 - Loading layer Frick Unit Service Area
2024-01-23 14:00:33.708 | INFO     | __main__:get_layer:11 - Loading layer APNs
2024-01-23 14:00:33.819 | INFO     | __main__:get_layer:45 - Clipping layer APNs
2024-01-23 14:00:33.844 | INFO     | __main__:get_layer:11 - Loading layer Landowner Well